# Chapter 4: Particle Systems（パーティクルシステム）

この章では、多数の小さな粒子を使って火、煙、爆発などの自然現象をシミュレートする方法を学びます。

## 主要な概念

1. **パーティクル** - 位置、速度、ライフスパンを持つ個々の粒子
2. **エミッター** - パーティクルを生成・管理するシステム
3. **継承** - クラスの拡張と再利用
4. **ポリモーフィズム** - 異なる種類のパーティクルの統一的な扱い

## 例題 1: 単一のパーティクル

ライフスパンを持つ基本的なパーティクルです。

In [ ]:
let particle;

function setup() {
  createCanvas(400, 400);
  particle = new Particle(200, 50);
}

function draw() {
  background(255);
  
  particle.update();
  particle.show();
  
  // パーティクルが死んだら再生成
  if (particle.isDead()) {
    particle = new Particle(200, 50);
  }
}

class Particle {
  constructor(x, y) {
    this.position = createVector(x, y);
    this.velocity = createVector(random(-1, 1), random(-2, 0));
    this.acceleration = createVector(0, 0.05);
    this.lifespan = 255;  // 透明度としても使用
  }
  
  update() {
    this.velocity.add(this.acceleration);
    this.position.add(this.velocity);
    this.lifespan -= 2;
  }
  
  show() {
    stroke(0, this.lifespan);
    strokeWeight(2);
    fill(127, this.lifespan);
    ellipse(this.position.x, this.position.y, 16, 16);
  }
  
  isDead() {
    return this.lifespan < 0;
  }
}

## 例題 2: パーティクルシステム（エミッター）

複数のパーティクルを生成・管理するエミッターです。

In [ ]:
let emitter;

function setup() {
  createCanvas(400, 400);
  emitter = new Emitter(width / 2, 50);
}

function draw() {
  background(255);
  
  emitter.addParticle();
  emitter.run();
}

class Emitter {
  constructor(x, y) {
    this.origin = createVector(x, y);
    this.particles = [];
  }
  
  addParticle() {
    this.particles.push(new Particle(this.origin.x, this.origin.y));
  }
  
  run() {
    // 逆順でループ（削除時にインデックスがずれないため）
    for (let i = this.particles.length - 1; i >= 0; i--) {
      let p = this.particles[i];
      p.update();
      p.show();
      if (p.isDead()) {
        this.particles.splice(i, 1);
      }
    }
  }
}

class Particle {
  constructor(x, y) {
    this.position = createVector(x, y);
    this.velocity = createVector(random(-1, 1), random(-2, 0));
    this.acceleration = createVector(0, 0.05);
    this.lifespan = 255;
  }
  
  update() {
    this.velocity.add(this.acceleration);
    this.position.add(this.velocity);
    this.lifespan -= 2;
  }
  
  show() {
    stroke(0, this.lifespan);
    strokeWeight(2);
    fill(127, this.lifespan);
    ellipse(this.position.x, this.position.y, 16, 16);
  }
  
  isDead() {
    return this.lifespan < 0;
  }
}

## 例題 3: 力を適用するパーティクル

重力と風の力をパーティクルに適用します。

In [ ]:
let emitter;

function setup() {
  createCanvas(400, 400);
  emitter = new Emitter(width / 2, 50);
}

function draw() {
  background(255);
  
  // 力を定義
  let gravity = createVector(0, 0.1);
  emitter.applyForce(gravity);
  
  // マウスが押されている間、風を適用
  if (mouseIsPressed) {
    let wind = createVector(0.2, 0);
    emitter.applyForce(wind);
  }
  
  emitter.addParticle();
  emitter.run();
}

class Emitter {
  constructor(x, y) {
    this.origin = createVector(x, y);
    this.particles = [];
  }
  
  addParticle() {
    this.particles.push(new Particle(this.origin.x, this.origin.y));
  }
  
  applyForce(force) {
    for (let p of this.particles) {
      p.applyForce(force);
    }
  }
  
  run() {
    for (let i = this.particles.length - 1; i >= 0; i--) {
      let p = this.particles[i];
      p.update();
      p.show();
      if (p.isDead()) {
        this.particles.splice(i, 1);
      }
    }
  }
}

class Particle {
  constructor(x, y) {
    this.position = createVector(x, y);
    this.velocity = createVector(random(-1, 1), random(-2, 0));
    this.acceleration = createVector(0, 0);
    this.lifespan = 255;
    this.mass = 1;
  }
  
  applyForce(force) {
    let f = p5.Vector.div(force, this.mass);
    this.acceleration.add(f);
  }
  
  update() {
    this.velocity.add(this.acceleration);
    this.position.add(this.velocity);
    this.acceleration.mult(0);
    this.lifespan -= 2;
  }
  
  show() {
    stroke(0, this.lifespan);
    strokeWeight(2);
    fill(127, this.lifespan);
    ellipse(this.position.x, this.position.y, 16, 16);
  }
  
  isDead() {
    return this.lifespan < 0;
  }
}

## 例題 4: 継承とポリモーフィズム

異なる見た目のパーティクルを継承を使って作成します。

In [ ]:
let emitter;

function setup() {
  createCanvas(400, 400);
  emitter = new Emitter(width / 2, 50);
}

function draw() {
  background(255);
  
  let gravity = createVector(0, 0.1);
  emitter.applyForce(gravity);
  
  emitter.addParticle();
  emitter.run();
}

class Emitter {
  constructor(x, y) {
    this.origin = createVector(x, y);
    this.particles = [];
  }
  
  addParticle() {
    // ランダムに2種類のパーティクルを追加
    if (random(1) < 0.5) {
      this.particles.push(new Particle(this.origin.x, this.origin.y));
    } else {
      this.particles.push(new Confetti(this.origin.x, this.origin.y));
    }
  }
  
  applyForce(force) {
    for (let p of this.particles) {
      p.applyForce(force);
    }
  }
  
  run() {
    for (let i = this.particles.length - 1; i >= 0; i--) {
      let p = this.particles[i];
      p.update();
      p.show();
      if (p.isDead()) {
        this.particles.splice(i, 1);
      }
    }
  }
}

// 基本パーティクル（円）
class Particle {
  constructor(x, y) {
    this.position = createVector(x, y);
    this.velocity = createVector(random(-1, 1), random(-2, 0));
    this.acceleration = createVector(0, 0);
    this.lifespan = 255;
  }
  
  applyForce(force) {
    this.acceleration.add(force);
  }
  
  update() {
    this.velocity.add(this.acceleration);
    this.position.add(this.velocity);
    this.acceleration.mult(0);
    this.lifespan -= 2;
  }
  
  show() {
    stroke(0, this.lifespan);
    strokeWeight(2);
    fill(127, this.lifespan);
    ellipse(this.position.x, this.position.y, 16, 16);
  }
  
  isDead() {
    return this.lifespan < 0;
  }
}

// コンフェティ（回転する四角）
class Confetti extends Particle {
  constructor(x, y) {
    super(x, y);
    this.angle = random(TWO_PI);
    this.angularVelocity = random(-0.2, 0.2);
  }
  
  update() {
    super.update();
    this.angle += this.angularVelocity;
  }
  
  show() {
    push();
    translate(this.position.x, this.position.y);
    rotate(this.angle);
    rectMode(CENTER);
    stroke(0, this.lifespan);
    strokeWeight(2);
    fill(200, 100, 100, this.lifespan);
    rect(0, 0, 12, 12);
    pop();
  }
}

## 例題 5: 複数のエミッター

クリックした場所に新しいエミッターを作成します。

In [ ]:
let emitters = [];

function setup() {
  createCanvas(400, 400);
}

function draw() {
  background(255);
  
  let gravity = createVector(0, 0.1);
  
  for (let emitter of emitters) {
    emitter.applyForce(gravity);
    emitter.addParticle();
    emitter.run();
  }
  
  // 説明テキスト
  fill(0);
  noStroke();
  text('Click to add emitter', 10, 20);
}

function mousePressed() {
  emitters.push(new Emitter(mouseX, mouseY));
}

class Emitter {
  constructor(x, y) {
    this.origin = createVector(x, y);
    this.particles = [];
  }
  
  addParticle() {
    this.particles.push(new Particle(this.origin.x, this.origin.y));
  }
  
  applyForce(force) {
    for (let p of this.particles) {
      p.applyForce(force);
    }
  }
  
  run() {
    for (let i = this.particles.length - 1; i >= 0; i--) {
      let p = this.particles[i];
      p.update();
      p.show();
      if (p.isDead()) {
        this.particles.splice(i, 1);
      }
    }
  }
}

class Particle {
  constructor(x, y) {
    this.position = createVector(x, y);
    this.velocity = createVector(random(-1, 1), random(-2, 0));
    this.acceleration = createVector(0, 0);
    this.lifespan = 255;
  }
  
  applyForce(force) {
    this.acceleration.add(force);
  }
  
  update() {
    this.velocity.add(this.acceleration);
    this.position.add(this.velocity);
    this.acceleration.mult(0);
    this.lifespan -= 2;
  }
  
  show() {
    let hue = map(this.lifespan, 0, 255, 200, 255);
    stroke(0, this.lifespan);
    fill(hue, 100, 100, this.lifespan);
    ellipse(this.position.x, this.position.y, 12, 12);
  }
  
  isDead() {
    return this.lifespan < 0;
  }
}

## 例題 6: 煙のようなパーティクル

上昇して広がる煙をシミュレートします。

In [ ]:
let emitter;

function setup() {
  createCanvas(400, 400);
  emitter = new SmokeEmitter(width / 2, height - 30);
}

function draw() {
  background(50);
  
  emitter.addParticle();
  emitter.run();
}

class SmokeEmitter {
  constructor(x, y) {
    this.origin = createVector(x, y);
    this.particles = [];
  }
  
  addParticle() {
    this.particles.push(new SmokeParticle(this.origin.x, this.origin.y));
  }
  
  run() {
    for (let i = this.particles.length - 1; i >= 0; i--) {
      let p = this.particles[i];
      p.update();
      p.show();
      if (p.isDead()) {
        this.particles.splice(i, 1);
      }
    }
  }
}

class SmokeParticle {
  constructor(x, y) {
    this.position = createVector(x, y);
    this.velocity = createVector(random(-0.5, 0.5), random(-2, -1));
    this.lifespan = 255;
    this.size = random(20, 40);
  }
  
  update() {
    // ノイズを使った横方向の揺らぎ
    this.velocity.x += random(-0.1, 0.1);
    this.position.add(this.velocity);
    this.lifespan -= 1.5;
    this.size += 0.5;  // 徐々に大きくなる
  }
  
  show() {
    noStroke();
    fill(200, this.lifespan * 0.5);
    ellipse(this.position.x, this.position.y, this.size, this.size);
  }
  
  isDead() {
    return this.lifespan < 0;
  }
}

## まとめ

### パーティクルシステムの構成
1. **Particle** - 個々の粒子（位置、速度、ライフスパン）
2. **Emitter** - 粒子を生成・管理するコンテナ

### 重要なテクニック
- **逆順ループ** - 配列から要素を削除する際のインデックスずれを防ぐ
- **継承（extends）** - 既存クラスを拡張して新しいパーティクルを作成
- **ポリモーフィズム** - 異なる種類のパーティクルを同じ配列で管理

### ライフスパンの活用
- 255から0へのカウントダウン
- 透明度（alpha）として使用
- 0以下で「死」と判定して削除